In [6]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

Датасет брать как показано ниже, каждый обьект это текст на русском и на английском. Я также дополнительно его токенизировал по предложениям для удобства

In [3]:
ds = load_dataset('VityaVitalich/wikihow_en_ru')

Generating train split:   0%|          | 0/35313 [00:00<?, ? examples/s]

In [9]:
ds['train'][0]

{'source': '. Эндометриоз — это заболевание, при котором клетки эндометрия разрастаются за пределами матки. Клетки могут разрастись вокруг матки и даже распространиться на другие участки брюшной полости. Основными симптомами этого заболевания являются сильные боли во время менструального цикла, которые могут продлиться на протяжении нескольких дней. К другим симптомам также можно отнести следующее:  боль во время секса; боль при испражнении или мочеиспускании, особенно во время менструального цикла; сильное кровотечение во время менструального цикла или в перерыве между циклами; бесплодие; прочие менее серьезные симптомы, вроде вздутия, диареи или запора, тошноты и усталости. Аденомиоз — это заболевание, при котором клетки эндометрия прорастают слишком глубоко в мышечную ткань матки. Это может привести к увеличению размеров матки, болям во время менструаций и прочим симптомам. У некоторых женщин при аденомиозе может не наблюдаться никаких симптомов, у других же может возникнуть следующ

# Эмбедер

Ниже пример эмбедера и того как его натравить на текст, чтобы получить эмбеддинг. Если правильно помню, кост у нас это косинусное расстояние

In [7]:
embedder = SentenceTransformer('sentence-transformers/LaBSE')

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

2_Dense/pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

In [11]:
sentences = ds['train'][0]['source_sentences'][:2] 
embeddings = embedder.encode(sentences)

In [14]:
embeddings.shape

(2, 768)

# Модель 
Ниже пример модели которую можно взять и использовать как переводчик

In [43]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = 'utrobinmv/t5_translate_en_ru_zh_small_1024'
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/809 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.6k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [45]:
prefix = 'translate to en: '

sentence_source = prefix + ds['train'][0]['source_sentences'][1]
sentence_target = ds['train'][0]['target_sentences'][1]
print(sentence_source)
print(sentence_target)

translate to en: Эндометриоз — это заболевание, при котором клетки эндометрия разрастаются за пределами матки.
Endometriosis is when endometrial tissue grows outside of the uterus.


In [47]:
inputs = tokenizer(sentence_source, return_tensors='pt')
target = tokenizer(sentence_target, return_tensors='pt')
print(inputs['input_ids'])
print(target['input_ids'])

tensor([[21809,    19,  1904,    31,  7711,   870,  5344,    48,  1934,  1248,
           140, 31113,     3,   138,  1297,  9814, 31686,  5344,   210,   344,
          7454, 11757,    79,  8364, 44995,     5,     1]])
tensor([[12055, 50978,  9113,    36,   266,   724, 50978,   167, 12933, 31059,
          2799,    11,     7,     4, 19258,   524,     5,     1]])


In [52]:
model_output = model.forward(input_ids=inputs['input_ids'], labels=target['input_ids']) # example of forward with labels

In [53]:
model_output.loss # classical LM loss

tensor(1.7059, grad_fn=<NllLossBackward0>)

In [54]:
model_output.logits # example of logits

tensor([[[ 3.1402,  2.8674, -0.4382,  ..., -0.0901, -0.3192,  0.0574],
         [-9.3730,  5.0923, -1.0458,  ..., -2.9596, -0.2955,  3.2580],
         [-3.0122,  4.9365,  1.2672,  ..., -2.3432, -0.4492, -0.4214],
         ...,
         [-2.4007, 14.5755,  2.8734,  ..., -4.6242,  2.0256, -3.8511],
         [ 5.1333, 14.6066,  2.7880,  ..., -0.7696, -0.8217, -1.5252],
         [ 2.3764, 23.6438,  3.2459,  ..., -2.5638,  0.7757, -1.5859]]],
       grad_fn=<UnsafeViewBackward0>)

In [48]:
generated_tokens = model.generate(**inputs) # example of generation. Further we would want to specify generation config

result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True) # decoding back to human readable text
print(result)

['Endometriosis is a disease in which endometrial cells grow outside the uterus.']
